# Welcome to ProgPy's Linear Model Example

The goal of this notebook is to instruct users on how to use ProgPy Model LinearModel.

This example shows the use of the LinearModel class, a subclass of PrognosticsModel for models that can be described as a linear time series, which can be defined by the following equations:



####    _<b>The State Equation<b>_:
$$
\frac{dx}{dt} = Ax + Bu + E
$$

#### _<b>The Output Equation<b>_:
$$
z = Cx + D
$$

#### _<b>The Event State Equation<b>_:
$$
es = Fx + G
$$

$x$ is `state`, $u$ is `input`, $z$ is `output`, and $es$ is `event state`

Linear Models are defined by creating a new model class that inherits from progpy's LinearModel class and defines the following properties:
* $A$: 2-D np.array[float], dimensions: n_states x n_states. <font color = 'teal'>The state transition matrix. It dictates how the current state affects the change in state dx/dt.</font>
* $B$: 2-D np.array[float], optional (zeros by default), dimensions: n_states x n_inputs. <font color = 'teal'>The input matrix. It dictates how the input affects the change in state dx/dt.</font>
* $C$: 2-D np.array[float], dimensions: n_outputs x n_states. The output matrix. <font color = 'teal'>It determines how the state variables contribute to the output.</font>
* $D$: 1-D np.array[float], optional (zeros by default), dimensions: n_outputs x 1. <font color = 'teal'>A constant term that can represent any biases or offsets in the output.</font>
* $E$: 1-D np.array[float], optional (zeros by default), dimensions: n_states x 1. <font color = 'teal'>A constant term, representing any external effects that are not captured by the state and input.</font>
* $F$: 2-D np.array[float], dimensions: n_es x n_states. <font color = 'teal'>The event state matrix, dictating how state variables contribute to the event state.</font>
* $G$: 1-D np.array[float], optional (zeros by default), dimensions: n_es x 1. <font color = 'teal'>A constant term that can represent any biases or offsets in the event state.</font>
* __inputs__:  list[str] - `input` keys
* __states__:  list[str] - `state` keys
* __outputs__: list[str] - `output` keys
* __events__:  list[str] - `event` keys

We will now utilize our LinearModel to model the classical physics problem throwing an object into the air! We can create a subclass of LinearModel which will be used to simulate an object thrown, which we will call the ThrownObject Class.


First, some definitions for our Model!

#### __Events__: (2)
* `falling: The object is falling`
* `impact: The object has hit the ground`

#### __Inputs/Loading__: (0)
* `None`

#### __States__: (2)
* `x: Position in space (m)`
* `v: Velocity in space (m/s)`

#### __Outputs/Measurements__: (1)
* `x: Position in space (m)`

Now, for our keyword arguments:

* <font color = green>__thrower_height : Optional, float__</font>
  * Height of the thrower (m). Default is 1.83 m
* <font color = green>__throwing_speed : Optional, float__</font>
  * Speed at which the ball is thrown (m/s). Default is 40 m/s

With our definitions, we can now create the ThrownObject Model.

First, we need to import the necessary packages.

In [ ]:
import numpy as np
from prog_models import LinearModel

Now we'll define some features of a ThrownObject LinearModel. Recall that all LinearModels follow a set of core equations and require some specific properties (see above). In the next step, we'll define our inputs, states, outputs, and events, along with the $A$, $C$, $E$, and $F$ values.

First, let's consider state transition. For an object thrown into the air without air resistance, velocity would decrease literally by -9.81 m/s^2 due to the effect of gravity, as described below:

 $$\frac{dv}{dt} = -9.81$$

 Position change is defined by velocity (v), as described below:
 
 $$\frac{dx}{dt} = v$$

Note: For the above equation x is position not state. Combining these equations to the model $\frac{dx}{dt}$ equation defined earlier yields the A and E matrix defined below. Note that there is no B defined because this model does not have an inputs.

In [ ]:
class ThrownObject(LinearModel):
    events = ['impact']
    inputs = []  
    states = ['x', 'v']
    outputs = ['x']
    
    A = np.array([[0, 1], [0, 0]])
    C = np.array([[1, 0]])
    E = np.array([[0], [-9.81]])
    F = None

Note that we defined our `A`, `C`, `E`, and `F` values to fit the dimensions that were stated at the beginning of the notebook! Since the parameter `F` is not optional, we have to explicitly set the value as __None__.

Next, we'll define some default parameters for our ThrownObject model.

In [ ]:
class ThrownObject(ThrownObject):  # Continue the ThrownObject class
    default_parameters = {
        'thrower_height': 1.83,
        'throwing_speed': 40,
        'g': 9.81,
    }

In the following cells, we'll define some class functions necessary to perform prognostics on the model.

The `initialize()` function sets the initial system state. Since we have defined the `x`and `v` values for our ThrownObject model to represent position and velocity in space, our initial values would be the thrower_height, and throwing_speed parameters, respectively.

In [ ]:
class ThrownObject(ThrownObject):
    def initialize(self, u=None, z=None):
        return self.StateContainer({
            'x': self.parameters['thrower_height'],
            'v': self.parameters['throwing_speed']
            })

For our `threshold_met()`, we define the function to return True for event 'falling' when our thrown object model has a velocity value of less than 0 (object is 'falling') and for event 'impact' when our thrown object has a distance from of the ground of less than or equal to 0 (object is on the ground, or has made 'impact').

`threshold_met()` returns a _dict_ of values, if each entry of the _dict_ is __True__, then our threshold has been met!

In [ ]:
class ThrownObject(ThrownObject):
    def threshold_met(self, x):
        return {
            'falling': x['v'] < 0,
            'impact': x['x'] <= 0
        }

Finally, for our `event_state()`, we will calculate the measurement of progress towards the events. We normalize our values such that they are in the range of 0 to 1, where 0 means the event has occurred.

In [ ]:
class ThrownObject(ThrownObject):
    def event_state(self, x): 
        x_max = x['x'] + np.square(x['v'])/(-self.parameters['g']*2)
        return {
            'falling': np.maximum(x['v']/self.parameters['throwing_speed'],0),
            'impact': np.maximum(x['x']/x_max,0) if x['v'] < 0 else 1
        }

With these functions created, we can now run our ThrownObject Model!

In this example, we will initialize our ThrownObject as `m`, and we'll use the `simulate_to_threshold()` function (more information about simulation: https://nasa.github.io/progpy/prog_models_guide.html#simulation) to simulate the movement of the thrown object in air.

In [ ]:
m = ThrownObject()
save = m.simulate_to_threshold(print = True, save_freq=1, threshold_keys='impact', dt=0.1)

__Note__: Because our model takes in no inputs, we have no need to actually define a future loading function! As a result, we are simply passing in an empty Input Container. However, for most models, there would be inputs, thus a need for a future loading function. For more information on future loading functions and when to use them, please refer to the ProgPy documentation [here](https://nasa.github.io/progpy/prog_models_guide.html#future-loading).

We'll also demonstrate how this looks plotted on a graph.

In [ ]:
import matplotlib.pyplot as plt
save.outputs.plot(title='generated model')
plt.show()

Notice that that plot resembles a parabola, which represents the position of the ball through space as time progresses!

#### Conclusion

In this example, we will initialize our ThrownObject as `m` and use the `simulate_to_threshold()` function (more information about simulation: https://nasa.github.io/progpy/prog_models_guide.html#simulation) to simulate the movement of the thrown object in air.